In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
study_name = "Easer"
study = optuna.create_study(
    study_name=study_name,
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-06 08:06:06,223] A new study created in memory with name: Easer


In [4]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

def objective(trial):
    topK = trial.suggest_int('topK', 0, 1000)
    l2_norm = trial.suggest_float('l2_norm', 1, 1e10, log=True)
    
    recommender = EASE_R_Recommender(data_train, verbose=False)
    recommender.fit(
        topK=topK if topK > 0 else None,
        l2_norm=l2_norm,
        normalize_matrix=False,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

c:\Users\Francesco1\Desktop\rcs\Recommenders\EASE_R\EASE_R_Recommender.py:73: RuntimeWarning: invalid value encountered in divide
  B = P / (-np.diag(P))
[W 2024-01-06 08:07:01,945] Trial 0 failed with parameters: {'topK': 461, 'l2_norm': 385627.4964763017} because of the following error: MemoryError((22222, 22222), dtype('float64')).
Traceback (most recent call last):
  File "c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Francesco1\AppData\Local\Temp\ipykernel_17072\2309083676.py", line 8, in objective
    recommender.fit(
  File "c:\Users\Francesco1\Desktop\rcs\Recommenders\EASE_R\EASE_R_Recommender.py", line 93, in fit
    self.W_sparse = check_matrix(B, format='npy', dtype=float)
  File "c:\Users\Francesco1\Desktop\rcs\Recommenders\Recommender_utils.py", line 45, in check_matrix
    return X.toarray().astype(dtype)
numpy.core._exceptions._ArrayMemoryError: 

MemoryError: Unable to allocate 3.68 GiB for an array with shape (22222, 22222) and data type float64

In [ ]:
study.best_params

In [ ]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender = EASE_R_Recommender(data_train + data_val, verbose=True)
recommender.fit(topK=72, l2_norm=36.47305040353163)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.39 min


In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)

TopPopRecommender: URM Detected 240 ( 1.9%) users with no interactions.
TopPopRecommender: URM Detected 114 ( 0.5%) items with no interactions.
